Connect to MongoDB Atlas
------------------------

In [1]:
import pymongo
import pprint

In [2]:
# Replace XXXX with your connection URI from the Atlas UI
uri = 'mongodb+srv://analytics:<password>@mflix.n69ze95.mongodb.net/test'
mc = pymongo.MongoClient(uri)
mflix = mc.mflix

List Indexes
-----------

In [3]:
# get list of indexes on movies collection
pprint.pprint(mflix.movies.index_information())

{'_id_': {'key': [('_id', 1)], 'v': 2}}


In [4]:
pprint.pprint(mflix.movies.find_one())

{'_id': ObjectId('644aceb41b724f94b632287e'),
 'cast': ['Carmencita'],
 'countries': ['USA'],
 'directors': ['William K.L. Dickson'],
 'fullPlot': 'Performing on what looks like a small wooden stage, wearing a '
             'dress with a hoop skirt and white high-heeled pumps, Carmencita '
             'does a dance with kicks and twirls, a smile always on her face.',
 'genres': ['Documentary', 'Short'],
 'imdb': {'id': 1, 'rating': 5.9, 'votes': 1032},
 'lastupdated': '2015-08-26 00:03:45.040000000',
 'plot': 'Performing on what looks like a small wooden stage, wearing a dress '
         'with a hoop skirt and white high-heeled pumps, Carmencita does a '
         'dance with kicks and twirls, a smile always on her face.',
 'poster': 'https://m.media-amazon.com/images/M/MV5BMjAzNDEwMzk3OV5BMl5BanBnXkFtZTcwOTk4OTM5Ng@@._V1_SX300.jpg',
 'rated': 'NOT RATED',
 'runtime': 1,
 'title': 'Carmencita',
 'type': 'movie',
 'year': 1894}


Explain a Query
--------------

In [5]:
explain = {
    "explain": {
        "find": "movies",
        "filter": {
            "tomatoes.viewer.numReviews": {"$gt": 10}
        },
    },
    "verbosity": "executionStats"
}
mflix.command(explain)

{'explainVersion': '1',
 'queryPlanner': {'namespace': 'mflix.movies',
  'indexFilterSet': False,
  'parsedQuery': {'tomatoes.viewer.numReviews': {'$gt': 10}},
  'queryHash': '412E8BA6',
  'planCacheKey': '412E8BA6',
  'maxIndexedOrSolutionsReached': False,
  'maxIndexedAndSolutionsReached': False,
  'maxScansToExplodeReached': False,
  'winningPlan': {'stage': 'COLLSCAN',
   'filter': {'tomatoes.viewer.numReviews': {'$gt': 10}},
   'direction': 'forward'},
  'rejectedPlans': []},
 'executionStats': {'executionSuccess': True,
  'nReturned': 0,
  'executionTimeMillis': 160,
  'totalKeysExamined': 0,
  'totalDocsExamined': 46014,
  'executionStages': {'stage': 'COLLSCAN',
   'filter': {'tomatoes.viewer.numReviews': {'$gt': 10}},
   'nReturned': 0,
   'executionTimeMillisEstimate': 70,
   'works': 46016,
   'advanced': 0,
   'needTime': 46015,
   'needYield': 0,
   'saveState': 46,
   'restoreState': 46,
   'isEOF': 1,
   'direction': 'forward',
   'docsExamined': 46014}},
 'command': {'f

Text Search vs Exact Match
-------------------------

In [6]:
filters = {"title": "Titanic"}
for m in mflix.movies.find(filters):
    pprint.pprint(m['title'])

'Titanic'
'Titanic'
'Titanic'


In [7]:
filters = { "$text": { 
    "$search": "titanic"
}} 
for m in mflix.movies.find(filters):
    pprint.pprint(m['title'])
    pprint.pprint(m['cast'])
    pprint.pprint(m.get('directors', ""))
    pprint.pprint("======")

OperationFailure: text index required for $text query, full error: {'ok': 0.0, 'errmsg': 'text index required for $text query', 'code': 27, 'codeName': 'IndexNotFound', '$clusterTime': {'clusterTime': Timestamp(1682844448, 1), 'signature': {'hash': b'A\x0bKu)Q\x8d\x10a\x8e:\x16cx\x11L\xc1\\\x7f\x8e', 'keyId': 7161371681292812292}}, 'operationTime': Timestamp(1682844448, 1)}

Create an Index
--------------

In [8]:
mflix.movies.create_index([("countries", pymongo.ASCENDING)])

'countries_1'